In [15]:
#https://qiskit.org/documentation/nature/tutorials/01_electronic_structure.html

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

large_molecule = True

if large_molecule:
    driver = PySCFDriver(
        atom="Co 6.844708500129253, 5.772690500054138, 5.0;O 8.228239500226193, 6.352208500094744, 5.0;C 9.29272350030078, 6.822059500127666, 5.0; O 10.323201500372985, 7.276646500159519, 5.0;",
        basis="sto3g",
        charge=1,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )
else:
    driver = PySCFDriver(
        atom="H 0 0 0; H 0 0 0.735",
        basis="sto3g",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )
    
def details(problem):
    print(problem)
    print(problem.num_particles)
    print(problem.num_spatial_orbitals)
    print(problem.hamiltonian.constants)

    hamiltonian = problem.hamiltonian

    coefficients = hamiltonian.electronic_integrals
    print(coefficients.alpha)

    second_q_op = hamiltonian.second_q_op()
    print(second_q_op)
    print(hamiltonian.nuclear_repulsion_energy)

full_problem = driver.run()
problem = full_problem
print("Full")
details(problem)

from qiskit_nature.second_q.transformers import FreezeCoreTransformer

fc_transformer = FreezeCoreTransformer()

fc_problem = fc_transformer.transform(full_problem)
print("Freezed 1")
details(fc_problem)

fc_transformer = FreezeCoreTransformer(remove_orbitals=range(13,33))

fc_problem = fc_transformer.transform(full_problem)
print(fc_problem.num_particles)
print(fc_problem.num_spatial_orbitals)

print("Freezed 2")
details(fc_problem)



Full
(24, 24)
33
{'nuclear_repulsion_energy': 197.72157981525953}
Polynomial Tensor
 "+-":
[[-3.65678187e+02  4.38891446e+00  5.03645547e-03 ... -8.64216460e-05
  -1.11958108e-02 -2.17674388e-02]
 [ 4.38891446e+00 -9.46993900e+01  3.54010127e-02 ...  1.88580260e-04
   6.64761640e-02  8.15048831e-02]
 [ 5.03645547e-03  3.54010127e-02 -9.28127920e+01 ... -1.70447593e-02
  -2.26209124e-02 -2.30132118e-02]
 ...
 [-8.64216460e-05  1.88580260e-04 -1.70447593e-02 ... -3.34482975e+01
   6.54138545e-03  3.56471590e-03]
 [-1.11958108e-02  6.64761640e-02 -2.26209124e-02 ...  6.54138545e-03
  -2.66433646e+01  2.16509650e+00]
 [-2.17674388e-02  8.15048831e-02 -2.30132118e-02 ...  3.56471590e-03
   2.16509650e+00 -1.71741642e+01]]
 "++--":
[[[[ 1.63742811e+01 -2.35019312e+00  9.67521084e-04 ...  4.87631557e-05
     6.46422225e-03  1.26125097e-02]
   [-2.35019312e+00  5.78799165e-01 -1.17447565e-04 ... -1.22008225e-05
    -1.63518385e-03 -3.18211387e-03]
   [ 9.67521084e-04 -1.17447565e-04  2.7401475

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Freezed 1
(12, 12)
21
{'nuclear_repulsion_energy': 197.72157981525953, 'FreezeCoreTransformer': -1621.754353637829}
Polynomial Tensor
 "+-":
[[-1.04523762e+01  6.00286953e-01  1.14305489e-01 -2.90016939e-13
  -4.77401387e-03  1.65935903e-01  1.98505205e-13  3.31932538e-03
  -7.66468526e-14 -3.32868525e-15 -1.08691251e-02  1.99312470e-01
  -1.31929419e-12 -2.42758049e-03  1.65751549e-01  2.05996873e-12
   2.46843107e-01  5.71038238e-01  9.74175103e-04  9.76162655e-02
  -6.75809839e-02]
 [ 6.00286953e-01 -1.02798057e+01 -1.65960428e-01  1.73822585e-13
   6.36771346e-03  3.99514603e-01 -2.37657186e-13  2.02969592e-03
   2.08854057e-13 -3.84400071e-13  2.23663506e-02 -1.47659435e-01
   7.02953985e-13  1.26607659e-03 -1.11125353e-01 -1.39387737e-12
  -1.47139810e-01 -1.86149348e-02 -1.04243535e-04  1.53662413e-01
   2.92618536e-01]
 [ 1.14305489e-01 -1.65960428e-01 -1.06454422e+01  8.36086476e-14
  -4.92277507e-04 -4.39186095e-01 -3.85366915e-13  9.02856722e-03
   5.74373203e-13 -3.85851455

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, NumPyMinimumEigensolverFactory
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter

solver = GroundStateEigensolver(
    QubitConverter(JordanWignerMapper()),
    NumPyMinimumEigensolverFactory(),
)

result = solver.solve(fc_problem)
print(result)



=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1748.797527054995
  - computed part:      -4.200937616367
  - FreezeCoreTransformer extracted energy part: -1744.596589438629
~ Nuclear repulsion energy (Hartree): 197.72157981526
> Total ground state energy (Hartree): -1551.075947239736
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [735.05621649  577.92708037  462.98290052]
 
  0: 
  * Electronic dipole moment (a.u.): [719.02259752  565.72510209  453.5342699]
    - computed part:      [34.26631092  25.40935398  18.89726125]
    - FreezeCoreTransformer extracted energy part: [684.7562866  540.31574811  434.63700865]
  > Dipole moment (a.u.): [16.03361897  12.20197828  9.44863062]  Total: 22.25402957
                 (debye): [40.75339058  31.01433231  24.01602128]  Total: 56.56409577
 
